In [26]:
import ollama
import gradio as gr
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
MODEL = "llama3.2"

## Short Story Creator

In [4]:
# Constants
#user prompt args
GENRES = ["Fiction", "Nonfiction", "Drama", "Poetry", "Fantasy", "Horror", "Mystery", "Science Fiction", "Suspense", "Women's fiction", "Supernatural/Paranormal", "Suspense", "Young adult"]
THEMES = ["Love", "Redemption", "Forgiveness", "Coming of age", "Revenge", "Good vs evil", "Bravery and hardship", "The power of social status","The destructive nature of love", "The fallibility of the human condition"]
WRITING_STYLES = ["Expository", "Narrative", "Descriptive", "Persuasive", "creative"]
TONES = ["Formal", "Formal", "Optimistic", "Worried", "Friendly", "Curious", "Assertive", "Encouraging"]

In [5]:
def get_story_user_prompt(genre, theme, style, tone):
    user_prompt = f"You are looking at genre: {genre}\n"
    user_prompt += f"with the theme: {theme}.\n"
    user_prompt += f"Author's writingStyle is: {style}.\n"
    user_prompt += f"Tone of the story is: {tone}."
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [8]:
def create_short_story(genre, theme, style, tone):
    response = ollama.chat(model=MODEL, messages=[
            {"role": "system", "content": story_system_prompt},
            {"role": "user", "content": get_story_user_prompt(genre, theme, style, tone)}
          ])
    return response['message']['content']

In [6]:
story_system_prompt = "You are an creative story writing assistant that takes input like genre, theme, or character type \
and generates a short story."

In [9]:
response = create_short_story("Fiction", "Bravery and hardship", "Creativity", "Encouraging")
print(response)

**The Ember of Resilience**

In the scorching desert, where dunes stretched like golden seas and sunbeams scorched the earth, a young nomad named Aisha traversed the unforgiving terrain. Her eyes, like two dark stars, shone bright with determination, for she was on a quest to fulfill an unyielding dream.

Aisha's village had been ravaged by a brutal sandstorm that left her family homeless and her heart shattered. The loss of her mother, who had taught her the ancient ways of their people, still lingered like an open wound. Yet, Aisha refused to let despair consume her. Instead, she channeled her grief into the flame of resilience that burned within.

As she walked, the blistering sun beat down upon her, relentless in its ferocity. The wind howled, threatening to snuff out the ember of hope that flickered within her chest. Yet Aisha persevered, fueled by the memories of her mother's words: "A brave heart is not one that shuns hardship, but one that finds strength in the face of adversit

In [10]:
# Gradio UI
gr.Interface(fn=create_short_story, inputs=[gr.Dropdown(GENRES, label="genre", info="select your story genre"),
                                gr.Dropdown(THEMES, label="theme", info="select your story theme"),
                                gr.Dropdown(WRITING_STYLES, label="style", info="select writingStyle"),
                                gr.Dropdown(TONES, label="tone", info="select the tone")
             ], outputs="textarea").launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


## Building Chatbot

In [18]:
# Constants
system_message = "You are an optimistic assistant and will respond to user queries with positivity"


In [19]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)

    stream = ollama.chat(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
         if 'message' in chunk and 'content' in chunk['message']:
            chunk_text = chunk['message']['content']
            response += chunk_text or ""
            yield response

In [25]:
# gradio UI
gr.ChatInterface(fn=chat,
                 type='messages',
                 chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Type Question Here", container=False, scale=3),
    title="helpful Assistant",
    description="Ask me a question and I will give you an helpful answer",
    theme="Glass",
    examples=["Will the sky fall today?", "were dinosaur existed before?", "Advantages of work life balance", "Is yes yes and a no a no"]).launch()

/Users/sivaram/Developer/SDS/SDS-CP019-local-ai-writing-assistant/.venv/lib/python3.12/site-packages/gradio/components/chatbot.py:242: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(
/Users/sivaram/Developer/SDS/SDS-CP019-local-ai-writing-assistant/.venv/lib/python3.12/site-packages/gradio/chat_interface.py:222: UserWarning: The type of the gr.Chatbot does not match the type of the gr.ChatInterface.The type of the gr.ChatInterface, 'messages', will be used.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


## Website Summarizer

In [27]:
 # A class to represent a Webpage

class Website:
    """
    A utility class to represent a Website that we have scraped
    """
    url: str
    title: str
    text: str

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [189]:
# Define our system prompt
system_prompt = "You are a creative writing assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond with key points."

In [191]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "The contents of this website is as follows; \
please provide a short, crisp summary of this website with key points. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [192]:
def get_urls_from_prompt(prompt):
    urls=[]
    if prompt is not None:
        x= prompt.split()
        for i in x:
            if i.find("https:")==0 or i.find("http:")==0:
                urls.append(i)
    return urls

In [193]:
def summarizer_bot(prompt, history):
    url = get_urls_from_prompt(prompt)
    try:
       prompt += user_prompt_for(Website(url[0]))
    except:
       prompt += "No URL provided"

    messages = [
            {"role":"assistant", "content":system_prompt},
            {"role":"user", "content": prompt}
        ]
    result = ollama.chat(model=MODEL, messages=messages, stream= True)
    response = ""
    for chunk in result:
        if 'message' in chunk and 'content' in chunk['message']:
            chunk_text = chunk['message']['content']
            response += chunk_text or ""
            yield response



In [195]:
gr.ChatInterface(
    fn=summarizer_bot,
    type="messages",
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Type Question Here", container=False, scale=3),
    title="Summarizer bot",
    description="Give me the link to a website for which you want a summary of the contents of this website.",
    theme="Glass"
    ).launch()

/Users/sivaram/Developer/SDS/SDS-CP019-local-ai-writing-assistant/.venv/lib/python3.12/site-packages/gradio/components/chatbot.py:242: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(
/Users/sivaram/Developer/SDS/SDS-CP019-local-ai-writing-assistant/.venv/lib/python3.12/site-packages/gradio/chat_interface.py:222: UserWarning: The type of the gr.Chatbot does not match the type of the gr.ChatInterface.The type of the gr.ChatInterface, 'messages', will be used.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7874

To create a public link, set `share=True` in `launch()`.
